Semi-automate solving Advent of Code using Google AI Studio

Environment

    pip3 install advent-of-code-data
    pip3 install 'advent-of-code-data[nb]'
    pip3 install beautifulsoup4
    pip3 install keyring
    pip3 install -q -U google-generativeai


In [1]:
from aocd.models import Puzzle
from bs4 import BeautifulSoup
import google.generativeai as genai
import keyring
import re

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
puzzle = Puzzle(year=2024, day=13)
puzzle.url

# https://aistudio.google.com/apikey
# keyring.set_password("palevich-chen-industries-aoc2024", "google-ai-studio", "YOUR_KEY_HERE")
genai.configure(api_key=keyring.get_password("palevich-chen-industries-aoc2024", "google-ai-studio"))
model_name = "gemini-exp-1206" # "gemini-2.0-flash-exp" "gemini-1.5-flash"
model = genai.GenerativeModel(model_name)

In [3]:
def puzzle_prose(puzzle, id):
    """id 0 means the prose before either part is answered.
    id 1 is the prose after the first part is answered
    id 2 is the prose after both parts are answered
    """
    def puzzle_prose_path(puzzle, id):
        if id == 0:
            return puzzle.prose0_path
        if id == 1:
            return puzzle.prose1_path
        if id == 2:
            return puzzle.prose2_path
    
    with open(puzzle_prose_path(puzzle, id), 'r') as file:
        text = file.read()
    
    soup = BeautifulSoup(text.split("<main>")[-1])
    return soup.text

In [4]:
prompt = """
You are an expert Python coder. You are participating in the "Advent of Code" programming contest.  The following is a puzzle description. Write expert Python code to solve the puzzle.

The puzzle has two parts. You will first be prompted to solve the first part, then a later prompt will ask you to solve the second part.

Read the puzzle input in the form of a text string from puzzle.input_data. Assign the puzzle answer to the property puzzle.answer_a for the first part. Assign the puzzle answer to puzzle.answer_b for the second part.

Only assign to puzzle.answer_a one time. Don't use puzzle.answer_a as a temporary variable or an accumulator.

For example if the puzzle is, "The input is a series of numbers, one per line. Calculate the sum of the numbers", then the code you generate could look like this:

def solve_a(input_data):
   return sum([int(line) for line in input_data.splitlines()])
puzzle.answer_a = solve_a(puzzle.input_data)

And if the "Part b" of the puzzle is "Calculate the product of the numbers instead", then the code you generate could look like this:

def solve_b(input_data):
  return prod([int(line) for line in input_data.splitlines()])
puzzle.answer_b = solve_b(puzzle.input_data)

Assume that the input is valid. Do not validate the input.

There may be only one part to the puzzle. If that is the case, only solve the first part.

Think carefully. It is important to get the correct answer and for the program to run quickly.
Write the python code without comments or explanation. Use short variable names. Use subroutines, lambdas, list comprehensions and logical boolean operators where it will make the code shorter.
Define a function named parse that takes the input string and returns the parsed input. 

"""

In [5]:
def try_to_solve(puzzle, part=0, tips=''):
    def parse_code(response):
        text = response.candidates[0].content.parts[0].text
        matches = re.findall(r"^```python\n(.*)```$", text, re.MULTILINE | re.DOTALL)
        if len(matches) != 1:
            return None
        code = matches[0]
        return code
    full_prompt = prompt + '\n\n' + puzzle_prose(puzzle, part) + '\n\n' + tips
    response = model.generate_content(full_prompt)
    return parse_code(response)

In [6]:
tips = """Tips: Some of the prize machines are not solvable. Ignore those machines.
Use split('\\n\\n') to separate the prize machine inputs.
Treat the prize machines as a system of linear equations. Solve them using Cramer's Rule.
"""

In [8]:
code_a = try_to_solve(puzzle, 0, tips)
print(code_a)
exec(code_a)
code_b = try_to_solve(puzzle, 1, tips)
print('# ----')
print(code_b)
exec(code_b)

import re

def parse(i):
    m = []
    for b in i.split('\n\n'):
        l = b.splitlines()
        a = [int(s) for s in re.findall(r'\d+', l[0])]
        b = [int(s) for s in re.findall(r'\d+', l[1])]
        p = [int(s) for s in re.findall(r'\d+', l[2])]
        m.append((a, b, p))
    return m

def solve(a, b, p):
    d = a[0] * b[1] - a[1] * b[0]
    if d == 0:
        return None
    da = p[0] * b[1] - p[1] * b[0]
    db = a[0] * p[1] - a[1] * p[0]
    na = da // d
    nb = db // d
    if na < 0 or nb < 0 or na > 100 or nb > 100:
        return None
    if na * a[0] + nb * b[0] != p[0] or na * a[1] + nb * b[1] != p[1]:
        return None
    return na * 3 + nb

def solve_a(i):
    c = 0
    t = 0
    for a, b, p in parse(i):
        s = solve(a, b, p)
        if s is not None:
            c += 1
            t += s
    return t if c > 0 else 0

puzzle.answer_a = solve_a(puzzle.input_data)

# ----
import re

def parse(i):
  m = []
  for b in i.split('\n\n'):
    l = b.splitlines()